# StarryNight

## Team Members
- 902011, Moleri Andrea, a.moleri@campus.unimib.it
- 856114, Costantini Davide, d.costantini6@campus.unimib.it
- 865939, Armani Filippo, f.armani1@campus.unimib.it

## Description of the Reference Domain and Objectives

The StarryNight project aims to construct a star classification system utilizing machine learning techniques acquired during the course of the Machine Learning curriculum. The primary objective is to demonstrate that stars adhere to a discernible pattern, specifically referred to as the Hertzsprung-Russell Diagram or HR-Diagram. This diagram serves as the basis for classifying stars by plotting their features, providing valuable insights into their characteristics.

## Design Choices for Dataset Creation, Hypotheses, and Assumptions

![](../Images/Spectral%20Class%20Diagram.png)

In crafting the dataset, we assume that stars exhibit identifiable patterns in their features, allowing for meaningful classification. The HR-Diagram will be employed as the guiding framework for the classification system. The dataset will encompass a range of features crucial for classification:

- **Absolute Temperature**: the absolute temperature of a celestial body, denoted by the symbol $T$, is measured in Kelvin $(K)$. It represents the temperature of the body on the Kelvin scale, an absolute temperature scale where $0K$ corresponds to absolute zero, the theoretical lowest temperature possible. Absolute temperature is a fundamental parameter in astrophysics and is crucial for understanding the thermodynamic characteristics of celestial objects.

- **Relative Luminosity**: relative luminosity, expressed as $L/L_o$, denotes the ratio of a celestial body's luminosity $(L)$ to the solar luminosity $(L_o)$. Luminosity is the total amount of energy radiated by the object per unit time. This dimensionless quantity provides insights into the intrinsic brightness of a celestial body relative to the Sun, serving as a key metric for comparative stellar analyses.

- **Relative Radius**: the relative radius, denoted by $R/R_o$, represents the ratio of a celestial body's radius $(R)$ to the solar radius $(R_o)$. This dimensionless parameter is essential in characterizing the size of celestial bodies, facilitating the comparison of their physical dimensions relative to the Sun, which serves as a standard reference.

- **Absolute Magnitude**: absolute magnitude $(M_v)$ is a measure of the intrinsic brightness of a celestial object as it would appear if placed at a standard distance of 10 parsecs from an observer. This parameter is defined on the logarithmic scale, and its determination involves corrections for both distance and interstellar extinction. Absolute magnitude enables astronomers to assess the true luminosity of celestial bodies independent of their varying distances from Earth.

- **Star Color**: the term "Star Color" refers to the visual perception of a celestial object's hue as observed from Earth. This qualitative characteristic is attributed to the dominant wavelengths emitted by the star's surface. Common colors include white, red, blue, yellow, and yellow-orange. The analysis of star color provides valuable information about the temperature and composition of a star's outer layers.

- **Spectral Class**: spectral class categorizes stars based on their spectral characteristics, primarily determined by the temperature of their surfaces. The spectral sequence, from hottest to coolest, is denoted by the letters $O, B, A, F, G, K$, and $M$. This classification system aids astronomers in classifying stars and understanding their fundamental properties, such as temperature, luminosity, and chemical composition.

- **Star Type**: star type refers to the broader categorization of celestial objects based on their evolutionary stage and physical characteristics. This classification includes Red Dwarfs, which are low-mass and long-lived stars; Brown Dwarfs, sub-stellar objects not massive enough to sustain nuclear fusion; White Dwarfs, remnants of low to medium-mass stars; Main Sequence stars, like our Sun, undergoing hydrogen fusion; SuperGiants, massive and luminous stars; and HyperGiants, the most massive and intrinsically bright stars in the stellar hierarchy. Understanding star types is crucial for comprehending the diverse life cycles of stars in the cosmos.


The designated target for our classification endeavors will be the **Star Type**. The following definitions apply:

- $Lo = 3.828 * 10^{26}~Watts$ _(Average Luminosity of the Sun)_
- $Ro = 6.9551 * 10^{8}~m$ _(Average Radius of the Sun)_

## Data Formatting and Analysis

Let's star by running the code snippet below, that is needed in order to import all the necessary dependencies for the project.

In [1]:
# Importing the necessary dependencies
import keras
import tensorflow
import pandas as pd
import numpy as np
import seaborn as sns
from keras.layers import Dense
import matplotlib.pyplot as plt
from keras.models import Sequential
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'tensorflow'

What follows is the preliminary formatting of the dataset, that ensures that all the considered instances have correct and well-formatted values. It is also important to make sure that the target column is a categorical data type, which will lead to better performances, not only in operations, but also in training our model.

In [ ]:
# Reading the dataset from a CSV file
df = pd.read_csv("StarsDataset.csv")

# Converting each color to uppercase for consistency, removing leading and 
# trailing whitespaces, and replacing spaces with hyphens to ensure uniformity
df["Star Color"] = [color.upper().strip().replace(" ", "-") for color in df["Star Color"]]

# Converting the "Star Color" column to a categorical data type
df["Star Color"] = df["Star Color"].astype("category")

# Printing the results of our formatting
df

Having confirmed the dimensions and main features of the dataframe, our attention is now directed towards understanding the distribution of values within each target class. The upcoming code snippet aims to underline the distribution of the number of instances associated with each target classes. The values within the target classes appear to be evenly distributed, presenting a balanced sampling across the dataset. After this consideration, we will also describe the dataframe for easier reference, including relevant statistical values such as means and percentiles.

In [ ]:
# Visualizing the distribution of the target variable using a pie chart
plt.pie(df["Star Type"].value_counts(), labels=["0", "1", "2", "3", "4", "5"], autopct="%1.1f%%")

# Adding a legend for clarity
plt.legend()

# Setting a title for the plot
plt.title("Distribution of The Target Variable")

# Displaying the pie chart
plt.show()

# Generating relevant statistics for the dataset
df.describe()

Let's begin by visually exploring the dataset through a normalized bar plot, with a focus on the star type as our target variable. Consequently, we will depict all numerical attributes in relation to the star type itself, providing insight into their distribution. It is essential to acknowledge that the numeric values are normalized on a scale between 0 and 1. This normalization ensures visibility, considering the substantial differences in orders of magnitude among the represented values.

In [ ]:
# Define a normalization function to scale a series between 0 and 1
def normalize(series):
    return (series - series.min()) / (series.max() - series.min())

# Extract unique star types from the "Star Type" column
starTypes = df["Star Type"].unique()

# Create a temporary DataFrame for normalization
temp_df = df.copy()

# Normalize selected attributes based on star type
temp_df["Temperature (K)"] = temp_df.groupby("Star Type")["Temperature (K)"].transform(normalize)
temp_df["Luminosity(L/Lo)"] = temp_df.groupby("Star Type")["Luminosity(L/Lo)"].transform(normalize)
temp_df["Radius(R/Ro)"] = temp_df.groupby("Star Type")["Radius(R/Ro)"].transform(normalize)
temp_df["Absolute Magnitude(Mv)"] = temp_df.groupby("Star Type")["Absolute Magnitude(Mv)"].transform(normalize)

# Calculate mean values for each normalized attribute grouped by star type
value_means = {
    "Temperature": temp_df.groupby("Star Type")["Temperature (K)"].mean(),
    "Luminosity": temp_df.groupby("Star Type")["Luminosity(L/Lo)"].mean(),
    "Radius": temp_df.groupby("Star Type")["Radius(R/Ro)"].mean(),
    "Absolute Magnitude": temp_df.groupby("Star Type")["Absolute Magnitude(Mv)"].mean()
}

# Plotting the normalized attributes by star type
x = np.arange(len(starTypes))
width = 0.2
multiplier = 0

fig, ax = plt.subplots(figsize=(10, 6))  

# Iterating through attribute means and plotting bar charts
for attribute, measurement in value_means.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, measurement, width, label=attribute)
    multiplier += 1

# Setting plot labels and formatting
ax.set_ylabel('Normalized Values')
ax.set_title('Star Attributes by Type')
ax.set_xticks(x + width * (multiplier - 1) / 2)
ax.set_xticklabels(starTypes)
ax.legend(loc='upper left', ncols=3)
ax.set_ylim(0, 1)

# Displaying the legend inside the plot for better aesthetics
plt.legend(loc='best', bbox_to_anchor=(1.0, 1.0), title='Legend')  

# Display the plot
plt.show()

Following some iterative experimentation, a scatter plot is employed to visually represent a pair of attributes exhibiting linear separability. Specifically, our investigation identifies "Temperature (K)" and "Absolute Magnitude (Mv)" as the optimal attributes for this purpose within the dataframe. The resultant plot  affirms that there is no necessity for remapping the existing attribute space to a new one, as the selected attributes already effectively capture the desired linear separation.

In [ ]:
# Creating a scatter plot using Seaborn
ax = sns.scatterplot(
    x=df["Temperature (K)"].transform(normalize),
    y=df["Absolute Magnitude(Mv)"].transform(normalize),
    hue=df["Star Type"],
    palette="Set1"
)

# Setting labels for the x and y axes
ax.set(xlabel='Temperature (K)', ylabel='Absolute Magnitude (Mv)')

# Adding a title to the plot
plt.title('Normalized Temperature vs. Absolute Magnitude by Star Type')

# Adding a legend for better interpretation
ax.legend(loc='best')

# Displaying a grid on the plot for improved readability
ax.grid(True)

# Displaying the scatter plot
plt.show()

We then move on to Principal Component Analysis, starting with Variance Analysis

In [ ]:
# Selecting attributes for analysis, assuming the first four columns of the DataFrame
attributes = list(df.columns[:4])

# Standardizing the data using StandardScaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df[attributes])

# Applying Principal Component Analysis (PCA) on the scaled data
pca = PCA().fit(scaled_data)

# Define a list of colors for the bars
colors = ['red', 'blue', 'yellow', 'orange']

# Plotting the explained variance ratios for each principal component as bar chart
fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(attributes, pca.explained_variance_ratio_, color=colors, alpha=0.7)

# Plotting a line connecting the peaks
ax.plot(attributes, pca.explained_variance_ratio_, marker='o', color='black', label='Variance Ratio')

# Adding labels and title to the plot
plt.xlabel("Principal Components")
plt.ylabel("Variance Ratio")
plt.title("Variance Analysis")

# Adding legend
plt.legend()

# Adjusting layout to avoid label overlapping
plt.tight_layout()

# Displaying the plot
plt.show()

In [ ]:
pcs = pca.components_

fig, ax = plt.subplots(figsize=(8, 8))
ax.set_xlim([-1, 1])
ax.set_ylim([-1, 1])

for i, (x, y) in enumerate(zip(pcs[0, :], pcs[1, :])):
    # plot arrows from origin to (x, y)
    ax.quiver(0, 0, x, y, angles='xy', scale_units='xy', scale=1, color=plt.cm.Set1(i/len(pcs[0])), alpha=0.8)

    # display the label of the point
    ax.text(x, y, df.columns[i], fontsize='10')

# Add labels and title
ax.set_title("Principal Component Analysis")

# Add grid
ax.grid(True)

# Displaying the plot
plt.show()

### Variance Analysis Considerations 
- **Temperature $(K)$:** a Variance Ratio of approximately $0.6$ indicates a moderate variance compared to the reference population. This may suggest that stellar temperatures in the dataset are relatively homogeneous, but there are still significant differences.
- **Luminosity $(L/Lo)$:** a Variance Ratio of approximately $0.2$ indicates lower variance compared to temperatures, but it is still present. The luminosities of stars in the dataset appear to be more homogeneous compared to temperatures.
- **Radius $(R/Ro)$:** a Variance Ratio of about $0.1$ suggests low variance compared to the reference population for stellar radius. This may indicate that the radius of stars in the dataset are relatively similar to each other.
- **Absolute Magnitude $(Mv)$:** a Variance Ratio of approximately $0.05$ indicates very low variance compared to the reference population for absolute magnitude. This suggests that absolute magnitudes of stars in the dataset are very similar to each other.

In light of these considerations, it is our judgment that a reduction in the dataset's dimensionality is unwarranted. The numeric attributes currently at our disposal demonstrate adequate descriptiveness and non-redundancy in portraying the characteristics of the target class. Consequently, a decision has been reached to avoid the implementation of principal component analysis in this instance.

## Creation of train and test sets
Let's begin by dividing the Dataset into two categories: train and test

In [ ]:
# Renaming the 'Star Type' column to 'target' for clarity
df.rename(columns={'Star Type': 'target'}, inplace=True)

# Defining the feature columns for model training and testing
feature_columns = ["Temperature (K)", "Luminosity(L/Lo)", "Radius(R/Ro)", "Absolute Magnitude(Mv)"]

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df[feature_columns], df["target"], test_size=0.3, random_state=42)

Let's also standardize the features in the dataframe

In [ ]:
# Initializing a StandardScaler object for feature scaling
scaler = StandardScaler()

# Scaling the feature columns in the training set
X_train[feature_columns] = scaler.fit_transform(X_train[feature_columns])

# Scaling the feature columns in the testing set using the parameters from the training set
X_test[feature_columns] = scaler.transform(X_test[feature_columns])

# Displaying descriptive statistics for the scaled training set
X_train.describe()

## First Approach: Decision Trees
#### *TODO: Add Decision Tree Process*

## Second Approach: Neural Networks
#### *TODO: Add Neural Network Explanation*